In [7]:
import os
import pdb 
import numpy as np
from einops import rearrange
from vima.utils import *
from vima_bench import make,PARTITION_TO_SPECS
from vima import create_policy_from_ckpt
os.environ["TOKENIZERS_PARALLELISM"] = "true"
from glob import glob 
import pickle as pkl 
from PIL import Image 
from example import prepare_prompt,prepare_obs
import torch 
from torch.optim import Adam 
from vima.policy.vima_policy  import VIMAPolicy 
from pathlib import Path 
from torch.utils.tensorboard import SummaryWriter
from vima.trajectory.trajectory_dataset import TrajectoryLoader
from collections import defaultdict
from behavior_cloning_batched import * 

In [8]:
seed = 42
summary_dir ="/home/rlcorrea/CSE574_project_vima/model_logs_demo"
#Path to the trajectories 
traj_folder = "/scratch/rlcorrea/vima_v6/rearrange_then_restore/"

In [9]:
dl = TrajectoryLoader(
    traj_folder=traj_folder,
    traj_name="rearrange_then_restore",
    n_workers=2,
    batch_size=2,
    n_epochs=1,
    max_queue_size=20,
)
device = 'cuda:0'
#This are the  parameters for the  model used in the 2M configuration 
vima_config = {'embed_dim': 256, 'xf_n_layers': 1, 'sattn_n_heads': 8, 'xattn_n_heads': 8}
policy =  VIMAPolicy(**vima_config) 
weight_path = "/home/rlcorrea/CSE574_project_vima/model_weights/2M.ckpt"
ckpt = torch.load(weight_path,map_location=device) 
#load the pretrained model except for the policy agents weight. The action prediction is handeled by the cross attention_gpt 
policy.load_state_dict({k.replace('policy.',""):v for k,v in ckpt['state_dict'].items() if 'xattn_gpt' not in k},strict=False)
policy = policy.train()
writer,weight_path = init_summary_writter(summary_dir)
#make it so only the cross attention component is trainable 
for n,e in policy.named_parameters(): 
    e.requires_grad = False 
for n,e in policy.xattn_gpt.named_parameters(): 
    e.requires_grad = True 
policy = policy.to(device)
opti = Adam(policy.parameters(),lr=0.1)

In [11]:
for traj_ids,observations,actions ,prompt_infos ,trajectory_steps, in dl: 
    num_batches = len(prompt_infos)
    opti.zero_grad()
    total_loss =0 
    break 

In [19]:
from vima import utils as vima_utils 

In [38]:
prompt_infos[0][2].keys()

dict_keys(['cropped_img', 'bbox', 'mask'])

In [50]:
torch.stack(word_batch)

tensor([[ 419,  291, 5517, 4820,   12,   48, 5818,   11,  258, 7882,    5,    1],
        [ 419,  291, 5517, 4820,   12,   48, 5818,   11,  258, 7882,    5,    1]],
       device='cuda:0')

In [56]:
raw_prompts = [e[0] for e in prompt_infos] 
word_batch = torch.stack([e[1].to(device) for e in prompt_infos ]) 
image_batches =[e[2].to_torch_tensor(device=device) for e in prompt_infos]

In [57]:
image_batches

[DataDict {
   cropped_img: DataDict {
     front: tensor([[[[[  0,   0,  25,  ..., 129,   0,   0],
                [  0,   0,  25,  ..., 128,   0,   0],
                [  0,   0,  25,  ..., 129,   0,   0],
                ...,
                [  0,   0,  25,  ...,  25,   0,   0],
                [  0,   0,  25,  ...,  25,   0,   0],
                [  0,   0,  25,  ...,  25,   0,   0]],
     
               [[  0,   0,  25,  ...,  44,   0,   0],
                [  0,   0,  25,  ...,  44,   0,   0],
                [  0,   0,  25,  ...,  43,   0,   0],
                ...,
                [  0,   0,  25,  ...,  25,   0,   0],
                [  0,   0,  25,  ...,  25,   0,   0],
                [  0,   0,  25,  ...,  25,   0,   0]],
     
               [[  0,   0,  25,  ...,  45,   0,   0],
                [  0,   0,  25,  ...,  45,   0,   0],
                [  0,   0,  25,  ...,  45,   0,   0],
                ...,
                [  0,   0,  25,  ...,  25,   0,   0],
             

In [ ]:
The main challenge with this data is that there is no way for me to identify a way to forward 